In [1]:
import mmcv
from mmcv import Config
from mmdet.datasets import (build_dataloader, build_dataset,
                            replace_ImageToTensor)
from mmdet.models import build_detector
from mmdet.apis import single_gpu_test
from mmcv.runner import load_checkpoint
import os
from mmcv.parallel import MMDataParallel
import pandas as pd
from pandas import DataFrame
from pycocotools.coco import COCO
import numpy as np

In [2]:
classes = ("UNKNOWN", "General trash", "Paper", "Paper pack", "Metal", "Glass", 
           "Plastic", "Styrofoam", "Plastic bag", "Battery", "Clothing")
# config file 들고오기
cfg = Config.fromfile('/opt/ml/realcode/swinb/cascade_mask_rcnn_swin_base_patch4_window7_mstrain_480-800_giou_4conv1f_adamw_3x_coco.py')

PREFIX = '/opt/ml/input/data/'


epoch = 10

# dataset 바꾸기
cfg.data.train.classes = classes
cfg.data.train.img_prefix = PREFIX
cfg.data.train.ann_file = PREFIX + 'train.json'
cfg.data.train.pipeline[2]['img_scale'] = (512, 512)

cfg.data.val.classes = classes
cfg.data.val.img_prefix = PREFIX
cfg.data.val.ann_file = PREFIX + 'val.json'
cfg.data.val.pipeline[1]['img_scale'] = (512, 512)

cfg.data.test.classes = classes
cfg.data.test.img_prefix = PREFIX
cfg.data.test.ann_file = PREFIX + 'test.json'
cfg.data.test.pipeline[1]['img_scale'] = (512, 512)

cfg.data.samples_per_gpu = 4

cfg.seed=2020
cfg.gpu_ids = [0]
cfg.work_dir = '/opt/ml/realcode/work_dirs/swinb_plus'

# cfg.model.roi_head.bbox_head.num_classes = 11

cfg.optimizer_config.grad_clip = dict(max_norm=35, norm_type=2)
cfg.model.train_cfg = None

# checkpoint path
checkpoint_path = os.path.join(cfg.work_dir, f'epoch_{epoch}.pth')

In [3]:
dataset = build_dataset(cfg.data.test)
data_loader = build_dataloader(
        dataset,
        samples_per_gpu=1,
        workers_per_gpu=cfg.data.workers_per_gpu,
        dist=False,
        shuffle=False)

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


In [4]:
model = build_detector(cfg.model, test_cfg=cfg.get('test_cfg'))
checkpoint = load_checkpoint(model, checkpoint_path, map_location='cpu')

model.CLASSES = dataset.CLASSES
model = MMDataParallel(model.cuda(), device_ids=[0])

Use load_from_local loader


In [5]:
output = single_gpu_test(model, data_loader, show_score_thr=0.05)

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 837/837, 5.2 task/s, elapsed: 162s, ETA:     0s

In [13]:
print(output[0])

([array([], shape=(0, 5), dtype=float32), array([[164.97191  ,  75.90062  , 254.11137  , 166.203    ,   0.9444211]],
      dtype=float32), array([], shape=(0, 5), dtype=float32), array([], shape=(0, 5), dtype=float32), array([], shape=(0, 5), dtype=float32), array([], shape=(0, 5), dtype=float32), array([], shape=(0, 5), dtype=float32), array([], shape=(0, 5), dtype=float32), array([[172.6695   , 233.61838  , 461.31226  , 473.19327  ,   0.9996213]],
      dtype=float32), array([], shape=(0, 5), dtype=float32), array([], shape=(0, 5), dtype=float32)], [[], [{'size': [512, 512], 'counts': b'`bb25j?3N2M8I2M3N6J4L1O4L4L1O2N7I1O2N2N1O1O1O01O01O1O0010O000000000001O000000001O1O01O0000010O001O000001O010O010O01O0010O0100O01O0100O10O1O01O01000000O010O1000O101O1N102M7J3L5K>C7H6H[kP4'}], [], [], [], [], [], [], [{'size': [512, 512], 'counts': b'dif2l0Q?7K3N1N2N2O1N2N3L8Af0A6K4M1N2O1N2N2O1M4I7I7M2O2M1O2O1N2N2N2N2M2O2N2O0O2O1N101N2M3N2N1O2N101O0O2O0O2N3M2N1O2N2O0O101O001O1O1N3N1O1O001O0O100O2O0O2N1O

In [14]:
([array([], shape=(0, 5), dtype=float32), array([[1.6645169e+02, 7.4975540e+01, 2.5428650e+02, 1.6588290e+02,
        5.1220137e-01],
       [2.3323289e+02, 1.5759534e+02, 2.5182079e+02, 2.3958292e+02,
        1.3505103e-01],
       [0.0000000e+00, 3.3629037e+02, 7.4538223e+01, 4.5377475e+02,
        9.4806015e-02]], dtype=float32), array([[1.6658017e+02, 7.5110573e+01, 2.5426059e+02, 1.6588449e+02,
        4.6446812e-01],
       [1.2174674e+02, 1.2843819e+01, 3.6410468e+02, 2.3813507e+02,
        5.4376066e-02]], dtype=float32), array([], shape=(0, 5), dtype=float32), array([], shape=(0, 5), dtype=float32), array([], shape=(0, 5), dtype=float32), array([[0.0000000e+00, 3.3632404e+02, 7.4400795e+01, 4.5374854e+02,
        9.6029855e-02]], dtype=float32), array([], shape=(0, 5), dtype=float32), array([[172.79912   , 238.60167   , 463.40448   , 467.66803   ,
          0.99824154],
       [  0.        , 336.4362    ,  75.0018    , 453.81778   ,
          0.47004235]], dtype=float32), array([], shape=(0, 5), dtype=float32), array([], shape=(0, 5), dtype=float32)], [[], [{'size': [512, 512], 'counts': b'_bc29e?7J3N3M3M4L7J1N2N8H3N2M3M1O1O100O001O1O010O00000000001O001O0000001O01OO100000000000000O10001O00000O2O00010O001O0011O1N10O011N10O2O0000O010O1000O100000000O3M4L8I7H3M3M9F5L4JbkP4'}, {'size': [512, 512], 'counts': b'odd3?`?;F8I:E:G4M3N3M5NK5K6H9H9G8G5L4L5IQYR4'}, {'size': [512, 512], 'counts': b'\\<d1d<mNPDh1m;^NgCi1X<P1001O1O1O001N2O2N0O2N1O2O1N2O00001O0O1O101N10000O0100000O001O0010000O1N2O100O1O1O1O1O2N1N2O101M2O2N1O2O1M4M2O1M3N2N2O1N2N2M3M4M2M5L2L5K5K<_OXTk6'}], [{'size': [512, 512], 'counts': b']bc2?`?3N2N5K2N3M6J2N3M7I3M3N2M1O100O1O001O10O01O000000001O010O0000001O0000000000000000000O1000001O0000001O001O000010O01001N1000O1O10O2O0000O010O011N0100000001N2N3M2O=B4L2N8H5J6JbkP4'}, {'size': [512, 512], 'counts': b'hQY22m?2N1O2N2N2N1O2N101N101O1N2O2M2O1O1O1O1O2N1N3N2N1O001O1O001O1O0dAhNP>Y1mAjNR>V1nAjNR>W1lAjNT>V1kAkNU>V1iAkNW>_1000001O1O2N3M2N2N2N2N6J:F001O1O1N2O1N2N2N2M2M4D;N3M3M2N3O1O1O1N2O1O1O1O1O1O1O1O1O0O2O1O1O00PNeD\\OZ;e0eD\\OZ;f2O1O1O1O1O001O1O001O1O1O2N1OmMoDXOP;h0PEXOP;h0PEYOo:l200000QNYEeNg:Z1^EbNb:^1aE_N_:a1dE\\N\\:c1kEWNU:j1lETNT:m1kETNT:l1mESNS:n1lERNT:o1lEPNT:Q2kEoMU:R2kEmMU:T2jElMV:U2jEkMU:U2lEjMT:V2lEjMT:W2lEhMT:X2lEhMT:X2mEgMS:Z2lEfMT:Z2lEgMS:Y2mEgMS:Z2lEfMT:Z2lEgMS:Y2mEiMQ:W2oEkMo9U2QFmMm9T2RFZN`9f1_F_N]9a1cF`N\\9`1dFaN[9`1cFaN]9`1bFaN]9`1aFaN_9a1_F`N`9a1_F_N`9c1^F_Na9a1_F_Na9b1]F`N5nMk8d3nF^N5QNl8a3oF^N5QNl8b3oF\\N5RNl8b3PG[N3TNm8a3QGZN2UNm8b3RGWN1WNm8b3TGUNNZNn8a3VGSNL\\Nn8a3XGaLAT19kNm8`3ZG]LEU15nNl8`3[G[LEX12nNn8_3[GYLGZ10nNn8_3gGcMZOoNo8^3gGcMYOPOP9]3gGcMXOQOQ9\\3gGcMWOROR9[3gGcMWOROR9[3dGZLYOT10XOS9[3`GhM]OmNS9]3YGkMDgNT9c3nFlMMaNU9g5kFXJV9h5jFWJW9i5iFTJZ9l5gFQJ[9o5eFQJ[9o5eFPJ\\9o5;O1N2TOSFdKn9[4ZF[Ki9e4i0O1O1O1N2O1G9N2N2O100O1L4I7L4M3L4M3N2M3N2N2M4L3H8J6L4L4M3M3N2L4J6I8J5L4M3M4K5J`0]O\\_Z2'}], [], [], [], [{'size': [512, 512], 'counts': b'\\<b1f<TOaCj1[<o0N1002N001O1N2O1O0O2O0O2N101N2O1O00000O1O2N10000O100000O10O001O1O100O010N2O101N1O1N200O2N1N2O1O2N1O2N101N2M3N3N1M2O2N3N0O2N2M4M3L3M4M3K5K6K9AYTk6'}], [], [{'size': [512, 512], 'counts': b'gif2e0U?=G6J5K4M2N4L4K6J8G>B6K5L2N2N101N2N2M3M4M2N2N2N2N101N1O2N1N4M2N2N2N1O2O0O2N1O2N2N2N2N2N101N2N1O100O2O0O101N101N101O0O2O1N2N101N100O2O00001O1O1N2O1O1O001O001O00001O001O1O1O1O001O001O0000001O000000001O001O001O1O1O1O1O0000001O000000000000001O001O001O1O1O1O001O001O0000000000000000001O00000000001O0000001O001O001O1O001O00001O000000000001O0000000000000000000001O0001O000000000000000000001O00001O00010O00001O000001O00001O01O01O00001O0001O01O00000001O01O01O001O1O1O1O001O001O001O1O1O2N1O1O001O001O001O1O1O1O1O1O1O001O001O1O2N1O1O1O001O0O2O001N2O2M2N2N2O0O2O1N2N2N4K5H9oNo0K5L3RN`C8c<AiC2a<ASD2T<FTD3V<^OXhh0'}, {'size': [512, 512], 'counts': b'X<h1d<kNgCm1V<ZNcCg1\\<o0001O1O1O001N3N0O2N1O2O1O1N101O000O2O0O1O100O10000000O10O001O1O01000O1N2O100O2N1O1O1O2M2O100O2N1N3N101N2M3O1N2N2M3N2O2M1O2M4M2N3L4L4L4L3L7HZTk6'}], [], []])

NameError: name 'array' is not defined

In [6]:
prediction_strings = []
file_names = []
coco = COCO(cfg.data.test.ann_file)
imag_ids = coco.getImgIds()

class_num = 11
for i, out in enumerate(output):
    prediction_string = ''
    image_info = coco.loadImgs(coco.getImgIds(imgIds=i))[0]
    for j in range(class_num):
        for o in out[0][j]:
            prediction_string += str(j) + ' ' + str(o[4]) + ' ' + str(o[0]) + ' ' + str(o[1]) + ' ' + str(
                o[2]) + ' ' + str(o[3]) + ' '
        
    prediction_strings.append(prediction_string)
    file_names.append(image_info['file_name'])


submission = pd.DataFrame()
submission['PredictionString'] = prediction_strings
submission['image_id'] = file_names
submission.to_csv(os.path.join(cfg.work_dir, f'submission_{epoch}.csv'), index=None)
submission.head()

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


,PredictionString,image_id
0,1 0.2749946 165.01508 76.848434 254.94998 165....,batch_01_vt/0021.jpg
1,1 0.22447498 286.10718 342.68176 306.35754 374...,batch_01_vt/0028.jpg
2,1 0.99928373 69.84436 169.72957 373.46982 379....,batch_01_vt/0031.jpg
3,8 0.9995408 33.860916 121.08598 388.68024 377....,batch_01_vt/0032.jpg
4,1 0.99515957 389.45132 272.238 473.27335 344.5...,batch_01_vt/0070.jpg
